<a href="https://colab.research.google.com/github/shisyamoCat/morethan-log/blob/main/matching_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# シートを出力するか true = 0, false = 1
# 'rms管材王マッチングデータ（重複分）'シートの出力フラグ
is_duplicate = 0
# '商品番号'シートの出力フラグ
is_extract_number = 0
# '選択肢タイプ'シートの出力フラグ
is_option_type = 0
# '除外'シートの出力フラグ
is_exceptions = 0

In [2]:
# '/content/drive/MyDrive'がgoogleドライブを開いた際に表示されるマイドライブ
# ドライブをマウントするために初回１度だけ実行
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
import os
import re
import numpy as np
import pandas as pd
import time

In [4]:
# モジュールを保存するディレクトリのパス
module_dir = '/content/drive/MyDrive/matchingData/modules'

# ディレクトリが存在しない場合は作成
if not os.path.exists(module_dir):
    os.makedirs(module_dir)

# モジュールインストール先を指定してインストール
# 初回インストール後はコメントアウトorリムーブ
# !pip install --target=$module_dir xlsxwriter

# モジュールのパスをシステムパスに追加
sys.path.append(module_dir)

In [5]:
#------------------------------------------------------------------------
# 関数定義
#------------------------------------------------------------------------

# 除外を抽出する関数
def extract_exceptions(df):
    # 抜き出す条件の文字列（後ろに追加するだけで自動的にソート）
    target_strings = sorted(['15zd-eliturbo2002', '10221600'])

    # 連結後のデータの対象列（例えば、'商品管理番号（商品URL）'を想定）
    column_to_search = '商品管理番号（商品URL）'

    # 二分探索で部分一致を確認する関数
    def binary_search_contains_partial(search_value, target_list):
        for target in target_list:
            if target in search_value:  # 部分一致を確認
                return True
        return False

    # 連結後のデータから除外対象の行を抽出
    df_exceptions = df[df.apply(lambda row: binary_search_contains_partial(str(row[column_to_search]), target_strings), axis=1)]

    return df_exceptions

# 選択肢タイプを抽出する関数
def extract_option_type(df):
    # `商品管理番号（商品URL）` がエンプティではなく、`選択肢タイプ` がエンプティでない行を抽出
    df_option_type = df[
        df['商品管理番号（商品URL）'].notna() & df['選択肢タイプ'].notna()
    ]

    return df_option_type

# 商品番号を抽出する関数
def extract_item_number(df):
    # 商品番号列にデータが入っている行を抽出し、`商品管理番号（商品URL）` 列でソート
    df_with_item_number = df[df['商品番号'].notna()]
    df_with_item_number = df_with_item_number.sort_values(by='商品管理番号（商品URL）', ascending=True)

    return df_with_item_number

# マッチングデータを補完する関数
def data_matching(df, item_number):
    # 商品管理番号の列名を変数化
    product_url_column = '商品管理番号（商品URL）'

    # 抽出した行を元のデータから削除し、残りのデータをソート
    df = df[~df.index.isin(item_number.index)]
    df = df.sort_values(by=product_url_column, ascending=True)

    # 補完列名
    complements_columns = ['商品番号', 'バリエーション6選択肢定義']

    # complements_columns の範囲に基づいて列を取得
    try:
        update_columns = df.columns[df.columns.get_loc(complements_columns[0]):
                                    df.columns.get_loc(complements_columns[1]) + 1]
    except KeyError as e:
        print(f"列名が見つかりません: {e}")
        raise

    # dfの商品管理番号（商品URL）に基づいてitem_numberをソート・拡張
    df_product_urls = df[product_url_column].values
    item_number_sorted = item_number.set_index(product_url_column).loc[df_product_urls].reset_index()

    # 更新マスクを作成
    update_mask = item_number_sorted[update_columns[0]].notna().values

    # ベクトル化された更新
    for col in update_columns:
        df.loc[update_mask, col] = item_number_sorted.loc[update_mask, col].values

    return df

# 重複分を抽出する関数
def extract_duplicate(df):
    # '商品管理番号（商品URL）' 列で重複する行をピックアップ
    df_duplicate_rows = df[df.duplicated(subset='商品管理番号（商品URL）', keep=False)]

    return df_duplicate_rows

def convert_to_numeric_if_possible(value):
    try:
        # 数値に変換可能な場合は変換
        return pd.to_numeric(value, errors='raise')
    except (ValueError, TypeError):
        # 数値に変換できない場合はそのまま返す
        return value

def convert_column_to_numeric_if_possible(df, column_name):
    # 指定された列を数値に変換可能であれば変換
    df[column_name] = df[column_name].apply(convert_to_numeric_if_possible)

    return df

# 数値と文字列の行を分離
def separate_num_and_str(df, column_name):
    # 数値化可能なら数値化
    df[column_name] = df[column_name].apply(convert_to_numeric_if_possible)

    # 数値と文字列でデータフレームを分ける
    numeric_rows = df[pd.to_numeric(df[column_name], errors='coerce').notna()]  # 数値行

    string_rows = df[pd.to_numeric(df[column_name], errors='coerce').isna()]    # 文字列行

    # 数値⇒文字列の順に結合
    df_separate = pd.concat([numeric_rows, string_rows])

    return df_separate

# 文字を変換
def convert_to_string(df, before, after):
    # applymapを使って全てのセルに対して処理を適用
    df = df.applymap(lambda x: x.replace(before, after) if isinstance(x, str) else x)
    return df

# DataFrameをExcelシートに書き込む関数
def write_to_excel(writer, df, sheet_name):
    if not df.empty:  # データフレームが空でないかチェック
        # NaNを空白に変換し、型を推定
        df = df.fillna('').infer_objects(copy=False)

        # xlsx形式のファイル内の各シートに書き出し
        df.to_excel(writer, index=False, sheet_name=sheet_name)

def display_elapsed_time(start_time):
    elapsed_time = time.time() - start_time
    hours = int(elapsed_time // 3600)
    minutes = int((elapsed_time % 3600) // 60)
    seconds = elapsed_time % 60

    print(f"\n処理にかかった時間: {hours}時間 {minutes}分 {seconds:.2f}秒")

In [6]:
# 処理開始時刻を取得
start_time_total = time.time()

In [7]:
#------------------------------------------------------------------------
# ファイルを全て読み込み、商品属性40~100をオミット
# データを加工して各シートに書き込むデータフレームを作成
#------------------------------------------------------------------------

# 処理開始時刻を取得
start_time = time.time()

# CSVファイルが保存されているディレクトリのパス
rms_dir_path = '/content/drive/MyDrive/matchingData/data/rms/'

# ディレクトリ内の全CSVファイルを取得
csv_files = [f for f in os.listdir(rms_dir_path) if f.endswith('.csv')]

# 削除したい列のリスト
pattern = re.compile(r'商品属性（(項目|値|単位)）([4-9][0-9]|100)$')

# 読み込んだデータを保存するデータフレーム
exceptions_df = pd.DataFrame()
option_type_df = pd.DataFrame()
extract_number_df = pd.DataFrame()
data_matching_df = pd.DataFrame()
duplicate_rows_df = pd.DataFrame()
filterd_df = pd.DataFrame()

# エンコーディングを指定
encoding_used = 'shift_jis'

# すべてのCSVファイルを格納するためのリスト
df_list = []

# ファイル読み込み
for i, file in enumerate(csv_files):
    # ファイルのパスを取得
    file_path = os.path.join(rms_dir_path, file)

    try:
        # RMSデータをデータフレームに読み込み
        df = pd.read_csv(file_path, encoding=encoding_used, low_memory=False, dtype=str)

        # データフレームをリストに追加
        df_list.append(df)

    except UnicodeDecodeError as e:
        print(f"エンコードエラーが発生しました: {file_path}")
        print(e)

# すべてのデータフレームを連結
df_combined = pd.concat(df_list, ignore_index=True)

# 削除対象の列を特定（正規表現にマッチする列）
columns_to_remove = [col for col in df_combined.columns if pattern.match(col)]

# 削除対象の列があれば削除
df_combined = df_combined.drop(columns=columns_to_remove, errors='ignore')

# [除外]を抽出
exceptions = extract_exceptions(df_combined)
# [除外]データフレームにデータを追加
exceptions_df = pd.concat([exceptions_df, exceptions], ignore_index=True)
# [マッチングデータ]から[除外]データを削除
df_filtered = df_combined[~df_combined.index.isin(exceptions.index)]

# [選択肢タイプ]を抽出
option_type = extract_option_type(df_filtered)
# [選択肢タイプ]データフレームにデータを追加
option_type_df = pd.concat([option_type_df, option_type], ignore_index=True)
# [マッチングデータ]から[選択肢タイプ]データを削除
df_filtered = df_filtered.drop(option_type.index)

# [商品番号]を抽出
extract_number = extract_item_number(df_filtered)
# [商品番号]データフレームにデータを追加
extract_number_df = pd.concat([extract_number_df, extract_number], ignore_index=True)
# [マッチングデータ]から[商品番号]データを削除
df_filtered = df_filtered.drop(extract_number.index)

# [rms管材王マッチングデータ]を補完
complement_data_matching = data_matching(df_filtered, extract_number)
# データフレームにデータを追加
data_matching_df = pd.concat([data_matching_df, complement_data_matching], ignore_index=True)

# [rms管材王マッチングデータ（重複分）]を抽出
duplicate_rows = extract_duplicate(complement_data_matching)
# [rms管材王マッチングデータ（重複分）]データフレームにデータを追加
duplicate_rows_df = pd.concat([duplicate_rows_df, duplicate_rows], ignore_index=True)
# [マッチングデータ]から[rms管材王マッチングデータ（重複分）]データを削除
df_filtered = complement_data_matching.drop(duplicate_rows.index)

# 重複行を [マッチングデータ]データフレーム から削除
df_filtered = df_filtered[~df_filtered.index.isin(duplicate_rows.index)]

# [マッチングデータ]データフレームにマッチングデータを追加
filterd_df = pd.concat([filterd_df, df_filtered], ignore_index=True)

# '商品管理番号（商品URL）'列の値でソート
filterd_df = filterd_df.sort_values(by='商品管理番号（商品URL）', ascending=True)

# 経過時間を出力（秒単位）
display_elapsed_time(start_time)


処理にかかった時間: 0時間 1分 41.42秒


In [8]:
#------------------------------------------------------------------------
# 管材王のデータをdf_kanzaiに読み込み
#------------------------------------------------------------------------

# 処理開始時刻を取得
start_time = time.time()

import glob

# ディレクトリのパス
directory_path = '/content/drive/MyDrive/matchingData/data/kanzai/'

# .xlsxファイルのパスを取得
xlsx_files = glob.glob(directory_path + '*.xlsx')

# 空のリストにデータフレームを格納する
df_list = []

# 複数ファイルをループ処理
for i, file in enumerate(xlsx_files):
    # 最初のファイルはヘッダーを含めて読み込み
    df = pd.read_excel(file, dtype=str)

    # データフレームをリストに追加
    df_list.append(df)

# すべてのデータフレームを連結
df_kanzai = pd.concat(df_list, ignore_index=True)

# 変換する列のリスト
columns_to_convert = [
    'M30_TANK11', 'M30_TANK12', 'M30_TANK13', 'M30_TANK14',
    'M30_TANK15', 'M30_TANK16', 'M30_TANK2', 'M30_HACU1',
    'M30_KUBN1', 'M30_KUBN2', 'M30_KUBN3', 'M30_KUBN4',
    'M30_KUBN5', 'M30_SURY1', 'M30_SURY2', 'M30_SURY3',
    'M30_BIRI1', 'M30_BIRI2'
]

# 特定の列をfloat型に変換し、.0のみ丸める
for col in columns_to_convert:
    if col in df_kanzai.columns:
        # float型に変換
        df_kanzai[col] = df_kanzai[col].astype(float)

# .0の値を整数に変換し、.1～.9はそのままにする
df_kanzai.replace({.0: 0}, inplace=True)

# 経過時間を出力（秒単位）
display_elapsed_time(start_time)


処理にかかった時間: 0時間 3分 5.42秒


In [9]:
#------------------------------------------------------------------------
# 管材王のデータヘッダーをfilterd_dfに設定
#------------------------------------------------------------------------

# 処理開始時刻を取得
start_time = time.time()

# 現在のcolumnsの長さを確認
current_columns_length = len(filterd_df.columns)

# 必要な列数
required_columns = 453

new_headers = list(df_kanzai.columns)

if current_columns_length > required_columns:
    # 列数が多すぎる場合、余分な列を削除
    filterd_df = filterd_df.iloc[:, :required_columns]
elif current_columns_length < required_columns:
    # 列数が足りない場合、空の列を一度に追加する
    columns_to_add = required_columns - current_columns_length
    empty_columns = pd.DataFrame(pd.NA, index=filterd_df.index, columns=[f'Empty_{i+1}' for i in range(columns_to_add)])
    filterd_df = pd.concat([filterd_df, empty_columns], axis=1)

# 新しいヘッダーリストを作成
new_column_list = list(filterd_df.columns[:375]) + [''] * (required_columns - 375 - len(new_headers)) + new_headers

# 新しいヘッダーを設定
filterd_df.columns = new_column_list

print("ヘッダーが設定されました。")

# 経過時間を出力（秒単位）
display_elapsed_time(start_time)

ヘッダーが設定されました。

処理にかかった時間: 0時間 0分 1.55秒


In [10]:
#------------------------------------------------------------------------
# 管材王のデータをRMSデータにコンバイン
#------------------------------------------------------------------------

# 処理開始時刻を取得
start_time = time.time()

# 文字列化する列名を指定
column_name = '商品管理番号（商品URL）'

# df_kanzaiのM30_CODE3列をすべて文字列に変換し、NaN値を処理
df_kanzai['M30_CODE3'] = df_kanzai['M30_CODE3'].fillna('').astype(str)

# df_kanzaiを'M30_CODE3'列でソートし、インデックスをリセット
df_kanzai_sorted = df_kanzai.sort_values('M30_CODE3').reset_index(drop=True)

# 二分探索のための関数を定義
def binary_search(arr, x):
    low = 0
    high = len(arr) - 1
    mid = 0

    while low <= high:
        mid = (high + low) // 2
        if arr[mid] < x:
            low = mid + 1
        elif arr[mid] > x:
            high = mid - 1
        else:
            return mid
    return -1  # 要素が見つからない場合

# M30_CODE3列の値を配列として取得
m30_codes = df_kanzai_sorted['M30_CODE3'].values

# filterd_dfの'商品管理番号（商品URL）'列の値を文字列に書き換え
filterd_df[column_name] = filterd_df[column_name].astype(str)

# '商品管理番号（商品URL）'列をベースにソート
filterd_df.sort_values(by=column_name, ascending=True)

# NumPy配列に変換
kanzai_values = df_kanzai_sorted.values

# デフォルトを '#N/A' で埋めるために、dtype=object を指定
na_filled_array = np.full((filterd_df.shape[0], df_kanzai_sorted.shape[1]), '#N/A', dtype=object)

for idx, value in enumerate(filterd_df[column_name]):
    matched_index = binary_search(m30_codes, value)
    if matched_index != -1:
        na_filled_array[idx] = kanzai_values[matched_index]

# 一括でfilterd_dfの416列目以降を更新
filterd_df.iloc[:, 416:416 + df_kanzai_sorted.shape[1]] = na_filled_array

# データフレーム内の数値をint型に変換
filterd_df.iloc[:, :375] = filterd_df.iloc[:, :375].apply(convert_to_numeric_if_possible)

# 変更を確認
print("データがコピーされました。")

# 経過時間を出力（秒単位）
display_elapsed_time(start_time)

データがコピーされました。

処理にかかった時間: 0時間 0分 15.16秒


In [ ]:
#------------------------------------------------------------------------
# RMSデータ変換、文字列⇒数値したのちソート
#------------------------------------------------------------------------

# 処理開始時刻を取得
start_time = time.time()

# 〜 (Unicode: 12316) を ～ (Unicode: 65374) に統一
filterd_df = convert_to_string(filterd_df, '〜', '～')

# '£' を '￡' に変換
filterd_df = convert_to_string(filterd_df, '£', '￡')

# 'ポイント変倍率適用期間（開始日時）' と 'ポイント変倍率適用期間（終了日時）' の両方の列に対して
# '/' を '-' に変換
target_point_columns = ['ポイント変倍率適用期間（開始日時）', 'ポイント変倍率適用期間（終了日時）']
filterd_df[target_point_columns] = filterd_df[target_point_columns].apply(lambda x: x.str.replace('/', '-'))

# 数値⇒文字列の順にソート
filterd_df = separate_num_and_str(filterd_df, '商品管理番号（商品URL）')

# 当該列名リスト
target_description_columns = ['PC用商品説明文', 'スマートフォン用商品説明文']

# 375列目までに制限し、除外列以外を数値化可能なら変換
filterd_df.loc[:, filterd_df.columns[:375][~filterd_df.columns[:375].isin(target_description_columns)]] = \
    filterd_df.loc[:, filterd_df.columns[:375][~filterd_df.columns[:375].isin(target_description_columns)]].applymap(convert_to_numeric_if_possible)

def process_cell(x):
    # 文字列に変換
    x = str(x)
    # 半角「=」または全角「＝」で囲まれた部分を抽出
    match = re.search(r'[=＝](.+?)[=＝]', x)
    if match:
        content = match.group()
        # 半角の「=」を全角の「＝」に変換
        content = content.replace('=', '＝')
        # 「＝」で始まる場合、先頭にゼロ幅非接合子を追加
        if content.startswith('＝'):
            return f'\u200C{content}'
        else:
            return content
    else:
        return ''

for col in target_description_columns:
    filterd_df[col] = filterd_df[col].apply(process_cell)

# 経過時間を出力（秒単位）
display_elapsed_time(start_time)

<ipython-input-5-3b0e96b86e4f>:114: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(before, after) if isinstance(x, str) else x)


In [ ]:
#------------------------------------------------------------------------
# 各シートへ対象データを書き出し、ファイル作成
#------------------------------------------------------------------------

# 処理開始時刻を取得
start_time = time.time()

# 出力ファイルパス（xlsx形式）
output_file = '/content/drive/MyDrive/matchingData/data/matching_data.xlsx'

# ExcelWriterを使用して複数のシートに書き込む
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    # メインのデータを [rms管材王マッチングデータ] シートに保存
    write_to_excel(writer, filterd_df, 'rms管材王マッチングデータ')

    # df_filteredを [rms管材王マッチングデータ（重複分）] シートに保存
    if is_duplicate == 1:
        write_to_excel(writer, duplicate_rows_df, 'rms管材王マッチングデータ（重複分）')

    # `商品番号` がエンプティではない行を [商品番号] シートに保存
    if is_extract_number == 1:
        write_to_excel(writer, extract_number_df, '商品番号')

    # `商品管理番号（商品URL）` がエンプティではなく、`選択肢タイプ` がエンプティでない行を [選択肢タイプ] シートに保存
    if is_option_type == 1:
        write_to_excel(writer, option_type_df, '選択肢タイプ')

    # 抜き出した行を [例外] シートに保存
    if is_exceptions == 1:
        write_to_excel(writer, exceptions_df, '例外')

    # フォーマットを適用するためのワークブックとシートを取得
    workbook = writer.book
    sheets = writer.sheets

    # MSゴシックフォントのフォーマットを定義
    ms_gothic_format = workbook.add_format({'font_name': 'MS Gothic', 'font_size': 11})

    # 中央寄せのフォーマットを定義
    center_format = workbook.add_format({'align': 'center', 'valign': 'vcenter'})

    # [rms管材王マッチングデータ]シートに対してフォーマットを適用
    worksheet = sheets['rms管材王マッチングデータ']

    # データ範囲を取得（行数、列数）
    rows, cols = df.shape

    # #N/Aセルに対して中央寄せの条件付きフォーマットを適用
    worksheet.conditional_format(1, 0, rows, cols - 1,
                             {'type': 'text',
                              'criteria': 'containing',
                              'value': '#N/A',
                              'format': center_format})

    # フォーマットをデータ範囲に適用
    worksheet.set_column(0, cols - 1, None, ms_gothic_format)

print(f"\nデータを {output_file} に保存しました。")

# 経過時間を出力（秒単位）
display_elapsed_time(start_time)

In [ ]:
# 経過時間を出力（秒単位）
display_elapsed_time(start_time_total)